In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ds-bowl-labels/__notebook__.ipynb
/kaggle/input/ds-bowl-labels/__results__.html
/kaggle/input/ds-bowl-labels/df_counter
/kaggle/input/ds-bowl-labels/df_counter_ini
/kaggle/input/ds-bowl-labels/__output__.json
/kaggle/input/ds-bowl-labels/labels_created_ini
/kaggle/input/ds-bowl-labels/custom.css
/kaggle/input/data-science-bowl-2019/test.csv
/kaggle/input/data-science-bowl-2019/specs.csv
/kaggle/input/data-science-bowl-2019/train.csv
/kaggle/input/data-science-bowl-2019/train_labels.csv
/kaggle/input/data-science-bowl-2019/sample_submission.csv
/kaggle/input/temp-ds-bowl-2019/df_counter
/kaggle/input/temp-ds-bowl-2019/df_counter_ini
/kaggle/input/temp-ds-bowl-2019/labels_created_ini


In [2]:
import IPython

def display(*dfs):
    for df in dfs:
        IPython.display.display(df)

# display(df1, df2)

In [3]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
%time df = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv', engine='c')
labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')

CPU times: user 1min 4s, sys: 8.55 s, total: 1min 12s
Wall time: 1min 12s


In [5]:
df = reduce_mem_usage(df)
labels = reduce_mem_usage(labels)

Mem. usage decreased to 778.73 Mb (18.2% reduction)
Mem. usage decreased to  0.49 Mb (48.2% reduction)


In [6]:
df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [7]:
df.shape, labels.shape

((11341042, 11), (17690, 7))

In [8]:
# # Format and make date / hour features
df['timestamp'] = pd.to_datetime(df['timestamp'])
# df['date'] = df['timestamp'].dt.date
# df['hour'] = df['timestamp'].dt.hour
# df['weekday_name'] = df['timestamp'].dt.weekday_name
# # Same for test
# # test['timestamp'] = pd.to_datetime(test['timestamp'])
# # test['date'] = test['timestamp'].dt.date
# # test['hour'] = test['timestamp'].dt.hour
# # test['weekday_name'] = test['timestamp'].dt.weekday_name

In [9]:
import gc
gc.collect()

11

In [10]:
df_test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')

In [11]:
df_test.shape

(1156414, 11)

In [12]:
sample_subm = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
sample_subm.head()

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3


In [13]:
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
specs.head()

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [14]:
# df_train = df.copy()
# df = df_test

In [15]:
# df = df_train

# Create labels

In [16]:
def create_label_all(df, params_to_group, labels=None):
#     id_unique = labels.installation_id.unique()
#     print('Numer of unique installation_id:', id_unique.shape[0])
    
    if labels is not None:
        # labels_part = labels[labels.installation_id.isin(id_unique[:n])]
        labels_part = labels.copy()
        
    # df_ids = df[df.installation_id.isin(id_unique[:n])].reset_index()
    df_ids = df.reset_index(drop=True)
    assert df_ids.shape[0] == df.shape[0]
    
    extracted_event = df_ids[df_ids.type == 'Assessment']
    print('# unique ids ini:', df_ids.installation_id.unique().shape[0])
    print('# unique ids in assessments', extracted_event.installation_id.unique().shape[0])
    extracted_event['correct'] = np.where(extracted_event.event_data.str.find('"correct":true')!=-1, True, False)
    # print('Number of assessments with "correct" in event_data:', extracted_event.shape)

    df_ids.loc[:, 'correct'] = extracted_event.correct
    #df_ids.update()
    del extracted_event
    
    index_for_correct = ~df_ids.correct.isna()
#     df_ids[index_for_correct]

    df_counter = df_ids[index_for_correct]
    df_counter_wt_BM = df_counter[(df_counter.event_code == 4100) & (df_counter.title.str.find('Bird Measurer')==-1)]
    df_counter_BM = df_counter[(df_counter.event_code == 4110) & (df_counter.title.str.find('Bird Measurer')!=-1)]
    df_counter_ini = df_counter_wt_BM.append(df_counter_BM)
    
    df_counter_ini.sort_values(by='timestamp', inplace=True)  
    df_counter = df_counter_ini.groupby('game_session', as_index=False).agg(params_to_group)

    df_counter['accuracy'] = df_counter.correct/df_counter.event_code
    if labels is not None:
        df_check = df_counter.merge(labels_part, on='game_session', how='left')
    
    return df_counter_ini, df_counter, df_check

# df_counter_ini, df_counter, df_check = create_label_all(df, labels)

In [17]:
params_group_like_train_label =  {'correct': 'sum',
                                  'event_code': 'count',
                                  'installation_id': lambda x: x.iloc[0],
                                  'title': lambda x: x.iloc[0]}

In [18]:
download_or_not = False

In [19]:
# df_counter_dw = df_counter.copy()
# df_counter_ini_dw = df_counter_ini.copy()

In [20]:
if download_or_not:
    print('Reading feathers')
    df_counter_ini = pd.read_feather('/kaggle/input/temp-ds-bowl-2019/df_counter_ini')
    df_counter = pd.read_feather('/kaggle/input/temp-ds-bowl-2019/df_counter')
else: 
    print('Calculating')
    df_counter_ini, df_counter, df_check = create_label_all(df, params_group_like_train_label,
                                                            labels)
    
    print('Writing to feather')
    df_counter.correct = df_counter.correct.astype(int)
    df_counter_ini.reset_index().to_feather('df_counter_ini')
    df_counter.reset_index().to_feather('df_counter')
    

Calculating
# unique ids ini: 17000
# unique ids in assessments 4242


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


Writing to feather


In [21]:
df_counter.head()

,game_session,correct,event_code,installation_id,title,accuracy
0,00097cda27afb726,1,1,01bdd720,Mushroom Sorter (Assessment),1
1,000f68cff32664ef,0,6,3f0dca37,Chest Sorter (Assessment),0
2,0014403daadf67aa,1,4,29d1aaee,Bird Measurer (Assessment),0.25
3,0014daa1d3e26eb2,1,3,55fdf49f,Mushroom Sorter (Assessment),0.333333
4,001c49e9e9968dbe,1,2,19d4b097,Bird Measurer (Assessment),0.5


In [22]:
# import json 
# extracted_event_data = pd.io.json.json_normalize(df_ids.event_data.apply(json.loads))
# print(extracted_event_data.shape)

In [23]:
'New labels:', df_counter.shape[0] - labels.shape[0]

try:
    df_error = df_check[(np.abs(df_check.accuracy_x-df_check.accuracy_y) > 0.001) & 
                         (~df_check.accuracy_y.isna())]
    print('Number of error calculated accuracy:', df_error.shape[0])
    #df_error
except:
    pass

Number of error calculated accuracy: 0


# Data study

## Create initial labels (like in train_labels file)

In [24]:
display(labels.head(), df_counter.head())

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


,game_session,correct,event_code,installation_id,title,accuracy
0,00097cda27afb726,1,1,01bdd720,Mushroom Sorter (Assessment),1
1,000f68cff32664ef,0,6,3f0dca37,Chest Sorter (Assessment),0
2,0014403daadf67aa,1,4,29d1aaee,Bird Measurer (Assessment),0.25
3,0014daa1d3e26eb2,1,3,55fdf49f,Mushroom Sorter (Assessment),0.333333
4,001c49e9e9968dbe,1,2,19d4b097,Bird Measurer (Assessment),0.5


In [25]:
print('# unique ids with assessment:', df.installation_id.unique().shape)
print('# unique ids in train:', df_counter.installation_id.unique().shape)
print('# unique ids in train_labels:', labels.installation_id.unique().shape)
    

# unique ids with assessment: (17000,)
# unique ids in train: (3614,)
# unique ids in train_labels: (3614,)


In [26]:
def fix_view(df_counter):
    df_counter['num_incorrect'] = df_counter.event_code - df_counter.correct
    df_counter.rename(columns={'correct': 'num_correct', 'event_code': 'total_attempts'},
                      inplace=True)
    df_counter.reset_index(inplace=True, drop=True)
    
    acc_group_map = {1: 3, 0.5: 2, 0: 0}
    df_counter['accuracy_group'] = df_counter.accuracy.apply(lambda x: acc_group_map[x] \
                                                         if x >=0.5 or x ==0 else 1)    
    return df_counter

In [27]:
df_counter = fix_view(df_counter)

columns_use = list(labels.columns)
columns_use.insert(5, 'total_attempts')
df_counter = df_counter[columns_use]

display(df_counter.head(), labels.head())

,game_session,installation_id,title,num_correct,num_incorrect,total_attempts,accuracy,accuracy_group
0,00097cda27afb726,01bdd720,Mushroom Sorter (Assessment),1,0,1,1,3
1,000f68cff32664ef,3f0dca37,Chest Sorter (Assessment),0,6,6,0,0
2,0014403daadf67aa,29d1aaee,Bird Measurer (Assessment),1,3,4,0.25,1
3,0014daa1d3e26eb2,55fdf49f,Mushroom Sorter (Assessment),1,2,3,0.333333,1
4,001c49e9e9968dbe,19d4b097,Bird Measurer (Assessment),1,1,2,0.5,2


,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [28]:
temp = df_counter.merge(labels[['accuracy', 'accuracy_group', 'game_session']],
                        how='left', on='game_session')
temp[temp.accuracy_group_x != temp.accuracy_group_y]

,game_session,installation_id,title,num_correct,num_incorrect,total_attempts,accuracy_x,accuracy_group_x,accuracy_y,accuracy_group_y


### How is samples looks like ?

In [29]:
def sequence_correct_attempt_by_game_session(df_counter_ini=df_counter_ini):
    df_counter_ini['correct_str'] = df_counter_ini.correct.astype(int).astype(str)
    to_add = df_counter_ini.groupby('game_session').agg({'correct_str': 'sum'})
    return to_add

In [30]:
#df_counter.reset_index(inplace=True)
to_add = sequence_correct_attempt_by_game_session(df_counter_ini)
assert to_add.shape[0], df_counter_download.shape[0]

df_counter = df_counter.merge(to_add, how='left', on='game_session')
df_counter

,game_session,installation_id,title,num_correct,num_incorrect,total_attempts,accuracy,accuracy_group,correct_str
0,00097cda27afb726,01bdd720,Mushroom Sorter (Assessment),1,0,1,1,3,1
1,000f68cff32664ef,3f0dca37,Chest Sorter (Assessment),0,6,6,0,0,000000
2,0014403daadf67aa,29d1aaee,Bird Measurer (Assessment),1,3,4,0.25,1,0001
3,0014daa1d3e26eb2,55fdf49f,Mushroom Sorter (Assessment),1,2,3,0.333333,1,001
4,001c49e9e9968dbe,19d4b097,Bird Measurer (Assessment),1,1,2,0.5,2,01
...,...,...,...,...,...,...,...,...,...
17685,ffe9dd5aa0bf9231,53858b6a,Cauldron Filler (Assessment),1,2,3,0.333333,1,001
17686,fff20fa828fb1a94,5fb6ef75,Cart Balancer (Assessment),1,0,1,1,3,1
17687,fff441b089d7b1ef,54d91d26,Cauldron Filler (Assessment),1,0,1,1,3,1
17688,fff53215178ede64,4db1e993,Bird Measurer (Assessment),1,0,1,1,3,1


In [31]:
import re
def find_unnormal_sequences(df):
    template = r'^[01]0*$' # start with 0or1 and then only 0s
    df.correct_str = df_counter.correct_str.str[::-1] # reverse string; now last attempt is on 0 index
    #print(df.head())
    return df[~df.correct_str.str.contains(template, regex=True)].index

In [32]:
idx = find_unnormal_sequences(df_counter)
df_counter.loc[idx]

,game_session,installation_id,title,num_correct,num_incorrect,total_attempts,accuracy,accuracy_group,correct_str


# Write new labels

In [33]:
df_counter.to_feather('labels_created_ini')

# To labels with all data

In [34]:
# Check that for each game_session corresponds to only 1 world, 1 title and 1 type
df_counter_ini.groupby('game_session').agg({'world': lambda x: (x == x.iloc[0]).all(),
                                            'title': lambda x: (x == x.iloc[0]).all(),
                                            'type': lambda x: (x == x.iloc[0]).all()}).all(axis=0)

world    True
title    True
type     True
dtype: bool

In [35]:
get_last = lambda x: x.iloc[:-1]
params_group = {'world': lambda x: x.iloc[0],
                'type': lambda x: x.iloc[0],
                'timestamp': lambda x: x.iloc[-1]}
params_group.update(params_group_like_train_label)
params_group

{'world': <function __main__.<lambda>(x)>,
 'type': <function __main__.<lambda>(x)>,
 'timestamp': <function __main__.<lambda>(x)>,
 'correct': 'sum',
 'event_code': 'count',
 'installation_id': <function __main__.<lambda>(x)>,
 'title': <function __main__.<lambda>(x)>}

In [36]:
df_all_ini, df_all, check = create_label_all(df, params_group, labels)

# unique ids ini: 17000
# unique ids in assessments 4242


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [37]:
'New labels:', df_all.shape[0] - labels.shape[0]

try:
    df_error = check[(np.abs(check.accuracy_x-check.accuracy_y) > 0.001) & 
                         (~check.accuracy_y.isna())]
    print('Number of error calculated accuracy:', df_error.shape[0])
    #df_error
except:
    pass

Number of error calculated accuracy: 0


In [38]:
fix_view(df_all)

,game_session,world,type,timestamp,num_correct,total_attempts,installation_id,title,accuracy,num_incorrect,accuracy_group
0,00097cda27afb726,TREETOPCITY,Assessment,2019-09-15 16:37:00.433000+00:00,True,1,01bdd720,Mushroom Sorter (Assessment),1,0,3
1,000f68cff32664ef,CRYSTALCAVES,Assessment,2019-10-12 23:08:24.584000+00:00,0,6,3f0dca37,Chest Sorter (Assessment),0,6,0
2,0014403daadf67aa,TREETOPCITY,Assessment,2019-08-03 22:21:52.112000+00:00,1,4,29d1aaee,Bird Measurer (Assessment),0.25,3,1
3,0014daa1d3e26eb2,TREETOPCITY,Assessment,2019-09-24 22:29:32.703000+00:00,1,3,55fdf49f,Mushroom Sorter (Assessment),0.333333,2,1
4,001c49e9e9968dbe,TREETOPCITY,Assessment,2019-07-26 15:50:54.148000+00:00,1,2,19d4b097,Bird Measurer (Assessment),0.5,1,2
...,...,...,...,...,...,...,...,...,...,...,...
17685,ffe9dd5aa0bf9231,MAGMAPEAK,Assessment,2019-09-08 16:20:35.686000+00:00,1,3,53858b6a,Cauldron Filler (Assessment),0.333333,2,1
17686,fff20fa828fb1a94,CRYSTALCAVES,Assessment,2019-09-26 17:40:17.701000+00:00,True,1,5fb6ef75,Cart Balancer (Assessment),1,0,3
17687,fff441b089d7b1ef,MAGMAPEAK,Assessment,2019-08-04 15:48:28.131000+00:00,True,1,54d91d26,Cauldron Filler (Assessment),1,0,3
17688,fff53215178ede64,TREETOPCITY,Assessment,2019-10-13 02:12:25.493000+00:00,True,1,4db1e993,Bird Measurer (Assessment),1,0,3


In [39]:
print('Writing to feather')
df_all.num_correct = df_counter.num_correct.astype(int)
df_all.to_feather('df_counter')
df_all_ini.reset_index(drop=True).to_feather('df_counter_ini')


Writing to feather
